In [2]:

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import os
def convert(path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, cache_dir="D:\hugging-models"
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        device=device,
    )


    result = pipe(path)
    print(result)
    # os.remove(path)
    return result 


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
convert(r"F:\works\A important\A neurals\IEEE-PaperGen\example_audio.mp3")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


{'text': ' என்னதான் விளவாசியில் ஏறியந்தாலும், இவர்கள் கொடுக்குக்கும் அஞ்சுரோ பொரட்டா மோசுதான். அது மட்டும் இல்லாமல், இந்த மாதிரி முட்டையை அப்படியே ஒரே வாயில்', 'chunks': [{'timestamp': (0.0, 2.12), 'text': ' என்னதான் விளவாசியில் ஏறியந்தாலும்,'}, {'timestamp': (2.12, 4.54), 'text': ' இவர்கள் கொடுக்குக்கும் அஞ்சுரோ பொரட்டா மோசுதான்.'}, {'timestamp': (4.54, None), 'text': ' அது மட்டும் இல்லாமல், இந்த மாதிரி முட்டையை அப்படியே ஒரே வாயில்'}]}


{'text': ' என்னதான் விளவாசியில் ஏறியந்தாலும், இவர்கள் கொடுக்குக்கும் அஞ்சுரோ பொரட்டா மோசுதான். அது மட்டும் இல்லாமல், இந்த மாதிரி முட்டையை அப்படியே ஒரே வாயில்',
 'chunks': [{'timestamp': (0.0, 2.12),
   'text': ' என்னதான் விளவாசியில் ஏறியந்தாலும்,'},
  {'timestamp': (2.12, 4.54),
   'text': ' இவர்கள் கொடுக்குக்கும் அஞ்சுரோ பொரட்டா மோசுதான்.'},
  {'timestamp': (4.54, None),
   'text': ' அது மட்டும் இல்லாமல், இந்த மாதிரி முட்டையை அப்படியே ஒரே வாயில்'}]}